# NumPy Neural Network Implementation From Scratch
This Python script is a highly generalizable implementation of a Multi-Layer Perceptron (MLP) network.

- Activation functions and cost functions can be varied and new ones can be defined in the relevant section by also defining the corresponding derivative and adding them to their respective dictionaries in the same manner as the rest.
- Network weights can be initialized as one of two choices: uniform random weight initialization $U(-1,1)$ or He Uniform weight initialization.
- L2-Regularization can be applied in training.

To initialize a neural network (neural) instance, you need the following arguments:
- a list of layer sizes in order, starting from the number of neurons in the input layer;
- a list of activation functions in order starting from the first hidden layer; and
- (optional) a boolean to determine whether He Uniform weight initialization is applied. If True, He Uniform weight initialization will be applied to each layer, otherwise $U(-1,1)$ will be applied.

In [ ]:
# Get all dependencies
import numpy as np
import pandas as pd             # Used to load the data
import matplotlib.pyplot as plt # Used only to plot relevant graphs

# Define Cost Functions with Derivatives

The cost function is the function that ultimately allows us determine which parameter modifications lead to better predictions from our neural network. A key assumption with gradient descent is that the cost function is differentiable. Without this assumption, we may need to precompute the resultant cost after making a prospective modification to any given parameter, and then modify the parameter accordingly, which would be computationally expensive. Thus, we should choose a cost function for which the derivative can be found.

It will be more practical to define the loss functions here so below we will define some loss functions and their associated derivatives.

In [ ]:
def mse_loss(y, y_hat):
    return 0.5*(y_hat - y)**2

def mse_loss_d(y, y_hat):
    return (y_hat - y)

def binary_ce_loss(y, y_hat):
    '''
    This function returns the binary cross entropy loss of two ndarrays with shape (n,m). Each row will consist of the loss values corresponding to a single 
    data point.

    Args:
        y       :   ndarray with shape (n,m) containing labels consisting of 0s and 1s. The number of rows n should correspond to the number of data points.
        y_hat   :   ndarray with shape (n,m) containing sigmoid value output predictions. The number of rows n should correspond to the number of data points.
    '''
    return y*np.log(y_hat+ 10**(-8)) + (1-y)*np.log(1-y_hat+ 10**(-8))

def binary_ce_loss_d(y, y_hat):
    '''
    This function returns the derivative of the binary cross entropy loss of two ndarrays with shape (n,m). Each row will consist of the derivative of the 
    loss values corresponding to a single data point.

    Args:
        y       :   ndarray with shape (n,m) containing labels consisting of 0s and 1s. The number of rows n should correspond to the number of data points.
        y_hat   :   ndarray with shape (n,m) containing sigmoid value output predictions. The number of rows n should correspond to the number of data points.
    '''
    return y/(y_hat + 10**(-8)) + (1-y)/(y_hat - 1 - 10**(-8))

def categorical_ce_loss(y, y_hat):
    return -y*np.log(y_hat+10**(-8))

def categorical_ce_loss_d(y, y_hat):
    return -y/(y_hat+10**(-8))

# Create dictionary to store loss functions and their derivatives
loss_function_dict = {
    'mean squared error': [mse_loss, mse_loss_d],
    'binary cross entropy': [binary_ce_loss, binary_ce_loss_d],
    'categorical cross entropy': [categorical_ce_loss, categorical_ce_loss_d]
}

[[0.         0.         0.         0.         0.        ]
 [1.50466654 0.2048005  2.07643925 1.13485422 0.41576086]]
[[-0.         -0.         -0.         -0.         -0.        ]
 [-4.50265193 -1.2272802  -7.97601767 -3.11072003 -1.5155234 ]]


# Define Activation Functions with Derivatives

Activation functions are the reason neural networks are not simply linear models, thus we must include them in our neural network.

There is also a key assumption that is made with gradient descent - that the derivative of the activation functions exists. This assumption is allows us to use multivariate calculus to determine the derivative of each of the network's parameters with respect to the cost function. Without this assumption, we may need to precompute the resultant cost after making a prospective modification to any given parameter, and then modify the parameter accordingly, which would be computationally expensive. Thus, we should choose activation functions for which the derivative can be found.

Below we will define some activation functions and their associated derivatives.

In [ ]:
# Define activation functions and their derivatives
def linear(zz):
    '''
    This function returns the linear output values of a given input array. This is just the identity activation function.

    Args:
        zz  :   ndarray containing the pre-activation (or weighted sum) values.
    '''
    return zz

def linear_d(zz):   # Derivative
    '''
    This function returns the derivative of the linear output values of a given input array. 
    In other words, this function returns 1.0.
    
    Args:
        zz  :   ndarray containing the pre-activation (or weighted sum) values.
    '''
    return 1.0

def relu(zz):
    '''
    This function returns the ReLU values of a given input array.    
    
    Args:
        zz  :   ndarray containing the pre-activation (or weighted sum) values.
    '''
    return np.maximum(zz, 0)

def relu_d(zz):     # Derivative
    '''
    This function returns the derivative of the ReLU values of a given input array.    
    
    Args:
        zz  :   ndarray containing the pre-activation (or weighted sum) values.
    '''
    return (zz>=0)

# Here a Leaky ReLU function is defined. The constant "leak" will be the gradient of the output when the output is below 0.
leak = 0.1
def leaky_relu(zz):
    '''
    This function returns the Leaky ReLU values of a given input array. This should 
    be an alternative to the ReLU function since the ReLU function may cause gradients 
    to become permanently 0 when the weighted sum is below 0.

    Args:
        zz  :   ndarray containing the pre-activation (or weighted sum) values.
    '''
    return zz*(zz >= 0) + (leak*zz)*(1 - (zz >= 0))

def leaky_relu_d(zz):
    '''
    This function returns the derivative of the Leaky ReLU values of a given input array.    
    
    Args:
        zz  :   ndarray containing the pre-activation (or weighted sum) values.
    '''
    return (zz >= 0) + (leak)*(1 - (zz >= 0))


def sigmoid(zz):
    '''
    This function returns the sigmoid values of a given input array.    
    
    Args:
        zz  :   ndarray containing the pre-activation (or weighted sum) values.
    '''
    return 1/(1+np.exp(-zz))

def sigmoid_d(zz):  # Derivative
    '''
    This function returns the derivative of the sigmoid values of a given input array.    
    
    Args:
        zz  :   ndarray containing the pre-activation (or weighted sum) values.
    '''
    return np.exp(-zz)/(1+np.exp(-zz))**2

def softmax(zz):
    '''
    This function returns the softmax values of a given input array.
    The input should have shape (n,m) where n is the number of data rows and m is the number of output neurons.
    
    Args:
        zz  :   ndarray containing the pre-activation (or weighted sum) values. Should have shape (n,m) where 
                n is the number of data rows and m is the number of output neurons.
    '''
    zz = zz.reshape(zz.shape[0], -1)    # BEWARE: this will turn 1D arrays of length n into a column vector with shape (n, 1), so if you input a single neural network output, make sure it is already a 2D array
    zz_max = np.max(zz, axis = 1).reshape(zz.shape[0], -1)
    return np.exp(zz - zz_max)/np.sum(np.exp(zz - zz_max), axis = 1).reshape(-1, 1)

def softmax_d(zz):  # Derivative
    '''
    This function returns the derivative of the softmax values of a given input array.
    The input should have shape (n,m) where n is the number of data rows and m is the number of output neurons.
    
    Args:
        zz  :   ndarray containing the pre-activation (or weighted sum) values. Should have shape (n,m) where 
                n is the number of data rows and m is the number of output neurons.
    '''
    softmax_output = softmax(zz)
    
    # This is the jacobian matrix and it will be used differently to derivatives of the other functions since 
    # those functions were defined element-wise while the softmax is defined on a vector, i.e. the other 
    # activation functions go from R to R while softmax goes from R^m to R^m
    jacobian = np.array([[[(softmax_output[nn, jj]*(1-softmax_output[nn, jj]) if jj == ii else -softmax_output[nn, ii]*softmax_output[nn, jj]) for jj in range(zz.shape[1])] for ii in range(zz.shape[1])] for nn in range(zz.shape[0])])
    
    return jacobian

# Here an exponential activation function is defined. The output will be e^(k*zz) where k=exp_const
exp_const = 0.01
def exponential(zz):
    '''
    This function returns the exponential output values for a given input array.

    Args:
        zz  :   ndarray containing the pre-activation (or weighted sum) values.
    '''
    return np.exp(exp_const*zz)

def exponential_d(zz):
    '''
    This function returns the exponential output derivative values for a given input array.

    Args:
        zz  :   ndarray containing the pre-activation (or weighted sum) values.
    '''
    return exp_const*np.exp(exp_const*zz)

def tanh(zz):
    '''
    This function returns the hyperbolic tangent output values for a given input array.

    Args:
        zz  :   ndarray containing the pre-activation (or weighted sum) values.
    '''
    #zz_max = np.max(zz, axis = 1).reshape(zz.shape[0], -1)
    return (np.exp(zz) - np.exp(-zz))/(np.exp(zz) + np.exp(-zz))

def tanh_d(zz):
    '''
    This function returns the hyperbolic tangent output derivative values for a given input array.

    Args:
        zz  :   ndarray containing the pre-activation (or weighted sum) values.
    '''
    return (2/(np.exp(zz) + np.exp(-zz)))**2

# Here an exponential linear unit activation function is defined with alpha equal to alpha_elu
alpha_elu = 1
def elu(zz):
    '''
    This function returns the exponential linear unit output values for a given input array.

    Args:
        zz  :   ndarray containing the pre-activation (or weighted sum) values.
    '''
    return zz*(zz >= 0) + alpha_elu*(np.exp(zz) - 1)*(zz < 0)

def elu_d(zz):
    '''
    This function returns the exponential linear unit output derivative values for a given input array.

    Args:
        zz  :   ndarray containing the pre-activation (or weighted sum) values.
    '''
    return 1*(zz >= 0) + alpha_elu*np.exp(zz)*(zz < 0)

# Create dictionary to store activation functions with their derivatives
# This dict will store a list containing the activation function at index 0, and the derivative at index 1
activation_function_dict = {
    'linear': [linear, linear_d],
    'relu': [relu, relu_d],
    'leaky relu': [leaky_relu, leaky_relu_d],
    'sigmoid': [sigmoid, sigmoid_d],
    'softmax': [softmax, softmax_d],
    'exponential': [exponential, exponential_d],
    'tanh': [tanh, tanh_d],
    'elu': [elu, elu_d]
}

# Create Neural Network

We will initialize a neural network given the layer lengths and the activation functions. This will fully determine the shapes of the weight and bias arrays. The weights will be initialized randomly and the biases will be initialized to 0.



In [6]:
# Create a function to initialize parameters
def create_parameters(layer_dims):
    '''
    This function will input a list containing layer dimensions (including the input layer size), and will output 
    randomly intialized weights in the range [-1, 1) at index 0 of the output, and biases intialized to 0 at index 1 
    of the output.

    Args:
        layer_dims  :   A list-like object containing integers representing the number of neurons in each layer.
                        The number of input neurons should be included at the first index and all other indices 
                        should be ordered accordingly following. The input layer is considered "layer 0".
    '''
    weight_dict = {}
    bias_dict = {}
    for i in range(len(layer_dims) - 1):
        weight_dict[f'layer {i + 1}'] = (np.random.rand(layer_dims[i], layer_dims[i + 1]) - 0.5)*2
        bias_dict[f'layer {i + 1}'] = np.zeros((1, layer_dims[i + 1]), dtype = float)

    return weight_dict, bias_dict

In [ ]:
class neural:
    def __init__(self, layers, activation_function_list, he_uniform=False):
        '''
        Args:
            layers                      :   A list-like object containing integers representing the number of neurons in each layer.
                                            The number of input neurons should be included at the first index and all other indices 
                                            should be ordered accordingly following. The input layer is considered "layer 0".
            activation_function_list    :   A list-like object containing the sublists. The sublists contain the activation functions 
                                            at subindex 0 and contain the derivatives of the activation functions at subindex 1. The 
                                            sublist at index i contains activation function and derivative for layer i + 1, thus, 
                                            first element of activation function list corresponds to layer 1 of the neural network.
                                            activation_function_list should have 1 less element than the layers list.
            he_uniform                  :   A boolean determining whether or not the parameters should be initialized using a He Uniform 
                                            distribution.
        '''
        self.layers = layers
        self.weights, self.biases = create_parameters(layers)
        self.activation_functions = [activation_function_dict[act] for act in activation_function_list]
        self.activation_function_name_list = activation_function_list

        # Initialize arrays to store history
        self.cost_history = np.array([], dtype = float)
        self.accuracy_history = np.array([], dtype = float)
        self.cv_cost_history = np.array([], dtype = float)
        self.cv_accuracy_history = np.array([], dtype = float)
        self.weight_update_max_history = np.zeros(shape=(0, len(self.layers) - 1), dtype = float)
        self.weight_update_mean_history = np.zeros(shape=(0, len(self.layers) - 1), dtype = float)

        # Apply He Uniform Initialization to the parameters if selected
        if he_uniform:
            for i in range(len(self.layers) - 1, 0, -1):
                self.weights[f'layer {i}'] *= np.sqrt(6/self.weights[f'layer {i}'].shape[0])

        

    def __call__(self, input_layer):
        '''
        This function computes the output of the neural network given an input layer.

        Args:
            input_layer     :   ndarray with shape (n, p) where p=self.layers[0], i.e., input_layer has the same shape 
                                as an input of the actual neural network.
        '''
        x = input_layer
        for i in range(len(self.layers) - 1):
            x = np.matmul(x, self.weights[f'layer {i + 1}']) + self.biases[f'layer {i + 1}']
            x = self.activation_functions[i][0](x)
        return x



    def call_with_parameters(self, input_layer, weights, biases):
        '''
        This function computes the output of the neural network given an input layer, weights, and biases. The weights 
        do not necessarily have to be the same as the model's weights, but there must be the same number of weight 
        matrices and these weight matrices must have the same shape as the model's corresponding weight matrices. The 
        same is true for the biases.

        Args:
            input_layer     :   ndarray with shape (n, p) where p=self.layers[0], i.e., input_layer has the same shape 
                                as an input of the actual neural network.
            weights         :   dict containing keys which are layers matching the model's layers with values being weight matrices 
                                matching the dimensions of the weight matrices of the model.
            biases          :   dict containing keys which are layers matching the model's layers with values being bias matrices 
                                matching the dimensions of the bias matrices of the model.
        '''
        x = input_layer
        for i in range(len(self.layers) - 1):
            x = np.matmul(x, weights[f'layer {i + 1}']) + biases[f'layer {i + 1}']
            x = self.activation_functions[i][0](x)
        return x



    def accuracy(self, true_labels, input_layer):
        '''
        This function outputs the accuracy of the model given an input and a target output.

        Args:
            true_labels     :   ndarray with shape (n, q) where q=self.layers[-1], i.e., true_labels has the same shape 
                                as an output of the actual neural network.
            input_layer     :   ndarray with shape (n, p) where p=self.layers[0], i.e., input_layer has the same shape 
                                as an input of the actual neural network.
        '''
        # If the output layer is softmax, then the variable is categorical, so we find the argmax's and calculate
        if self.activation_function_name_list[len(self.layers) - 2] == 'softmax':
            # Compute the predictions of the network and find the argmax value for each row
            output_layer = self.__call__(input_layer)
            predictions = np.argmax(output_layer, axis = 1)

            # Find the argmax value for each row of the true labels
            true_labs = np.argmax(true_labels, axis = 1)

            # Create a boolean array and compute the accuracy
            acc = (predictions == true_labs)
            acc = np.sum(acc)/acc.shape[0]
            return acc
        
        # If the output layer has sigmoid neurons, then the rounded output layer vector must exactly match the true label vector
        elif self.activation_function_name_list[len(self.layers) - 2] == 'sigmoid':
            # Compute the predictions of the network and find the rounded values for each row
            output_layer = self.__call__(input_layer)
            predictions = np.round(output_layer)

            # Compute the accuracy via boolean calculations and return the final value
            acc = (predictions == true_labels) - 1
            acc = np.sum(acc, axis = 1)
            acc = (acc == 0)
            acc = np.sum(acc)/acc.shape[0]
            return acc
    


    def neuron_activations(self, input_layer):
        '''
        This function will output the neuron activations and pre-activations for each layer in the network.

        Args:
            input_layer     :   ndarray with shape (n, p) where p=self.layers[0], i.e., input_layer has the same shape 
                            as an input of the actual neural network.
        '''
        a_dict = {}     # Dictionary storing the neuron activations
        z_dict = {}     # Dictionary storing the pre-activation function neuron activations
        x = input_layer

        # Add the input layer to a_dict for future use
        a_dict['layer 0'] = x

        for i in range(len(self.layers) - 1):
            x = np.matmul(x, self.weights[f'layer {i + 1}']) + self.biases[f'layer {i + 1}']
            z_dict[f'layer {i + 1}'] = x
            x = self.activation_functions[i][0](x)
            a_dict[f'layer {i + 1}'] = x
        return a_dict, z_dict
    

    
    def derivatives(self, input_layer, true_labels, loss, n_training_set, lambd=0):
        '''
        This function computes the derivatives of the cost function w.r.t. each weight, bias, and neuron in the network 
        (except for the neurons corresponding to the input layer which is "layer 0").

        Args:
            input_layer     :   ndarray with shape (n, p) where p=self.layers[0], i.e., input_layer has the same shape 
                                as an input of the actual neural network.
            true_labels     :   ndarray with shape (n, q) where q=self.layers[-1], i.e., true_labels has the same shape 
                                as an output of the actual neural network.
            loss            :   The name of the type of loss to be used in the derivative calculation. For example, 
                                "mean squared error", "binary cross entropy", or "categorical cross entropy".
            lambd           :   Regularization constant. The dafault is 0 corresponding to no regularization.
            n_training_set  :   The total number of training examples in the training set.
        '''
        a_derivatives = {}      # Dictionary storing the derivatives of the cost w.r.t. the neurons for each layer
        z_derivatives = {}      # Dictionary storing the derivatives of the cost w.r.t. the weighted sums for specific layers with softmax activation functions
        w_derivatives = {}      # Dictionary storing the derivatives of the cost w.r.t. the weights for each layer
        b_derivatives = {}      # Dictionary storing the derivatives of the cost w.r.t. the biases for each layer

        # Calculate the neuron activations and preactivations
        a_dict, z_dict = self.neuron_activations(input_layer)

        # Calculate dJ/da for each neuron a in the output layer
        a_derivatives[f'layer {len(self.layers) - 1}'] = loss_function_dict[loss][1](true_labels, a_dict[f'layer {len(self.layers) - 1}'])

        # If the final layer is softmax, calculate dJ/dz using dJ/da and da/dz (the jacobian)
        final = len(self.layers) - 1
        if self.activation_function_name_list[final - 1] == 'softmax':
            # Get dJ/da for layer i
            dJ_da = a_derivatives[f'layer {final}'].reshape(input_layer.shape[0], a_derivatives[f'layer {final}'].shape[1], 1)

            # Get the da/dz (the jacobian matrix) for layer i
            da_dz = self.activation_functions[final - 1][1](z_dict[f'layer {final}'])

            # Multiply the dJ/da and da/dz to get a (n,p,p) matrix whose column k (of the last index) sums to dJ/dz_k, when the first index is fixed
            prod = dJ_da*da_dz
            prod = np.sum(prod, axis = 1)

            # Save these derivatives in the dictionary for weighted sum derivatives (z_derivatives)
            z_derivatives[f'layer {final}'] = prod

        # Calculate the derivative of the cost w.r.t. each neuron in the network
        for i in range(len(self.layers) - 2, 0, -1):
            # Find dJ/da via the regular method if layer i + 1 is not softmax
            if self.activation_function_name_list[i] != 'softmax':
                # Get dJ/da for each neuron a in layer i + 1
                dJ_da = a_derivatives[f'layer {i + 1}']

                # Calculate the g'(z) values
                g_prime_z = self.activation_functions[i][1](z_dict[f'layer {i + 1}'])

                # Find the element-wise product of g'(z) and dJ/da
                prod = (dJ_da*g_prime_z)

                # Reshape the product so that it can broadcasted with NumPy and be multiplied elementwise with the weight matrix
                prod = prod.reshape(input_layer.shape[0], 1, prod.shape[-1], order = 'C')

                # Multiply the product with the weight matrix - if the second index is taken to be the rows (and third index is 
                # taken to be columns), the sum of row j will be the derivative of the cost w.r.t. a_j in layer i
                prod = prod*self.weights[f'layer {i + 1}']

                # Sum the rows to get a column vector for each data point whose jth entry is the derivative of the cost w.r.t. a_j
                # in layer i
                prod = np.sum(prod, axis = -1)

                # Save these derivatives in the dictionary for neuron derivatives
                a_derivatives[f'layer {i}'] = prod
            
            # Find dJ/da via dJ/dz for layer i + 1 and dz/da if layer i + 1 is softmax
            elif self.activation_function_name_list[i] == 'softmax':
                # Get dJ/da for each neuron a in layer i + 1
                dJ_da = a_derivatives[f'layer {i + 1}']

                # Calculate dz_da where z is from layer i + 1 and a is from layer i
                # dJ/dz layer i + 1
                prod = z_derivatives[f'layer {i + 1}'].reshape(input_layer.shape[0], 1, z_derivatives[f'layer {i + 1}'].shape[-1], order = 'C')
                prod = prod*self.weights[f'layer {i + 1}']
                prod = np.sum(prod, axis = -1)

                # Save these derivatives in the dictionary for neuron derivatives
                a_derivatives[f'layer {i}'] = prod
            
            # If layer i is softmax, calculate dJ/dz for layer i using dJ/da for layer i and da/dz (the jacobian matrix from the softmax derivative) for layer i
            if self.activation_function_name_list[i - 1] == 'softmax':
                # Get dJ/da for layer i
                dJ_da = a_derivatives[f'layer {i}'].reshape(input_layer.shape[0], a_derivatives[f'layer {i}'].shape[1], 1)

                # Get the da/dz (the jacobian matrix) for layer i
                da_dz = self.activation_functions[i - 1][1](z_dict[f'layer {i}'])

                # Multiply the dJ/da and da/dz to get a (n,p,p) matrix whose column k (the last index) sums to dJ/dz_k, when the first index is fixed
                prod = dJ_da*da_dz
                prod = np.sum(prod, axis = 1)

                # Save these derivatives in the dictionary for weighted sum derivatives (z_derivatives)
                z_derivatives[f'layer {i}'] = prod

        # Calculate the derivative of the cost w.r.t. each weight in the network
        for i in range(len(self.layers) - 1, 0, -1):
            # Find dJ/dw and dJ/db via the regular method if layer i is not softmax
            if self.activation_function_name_list[i - 1] != 'softmax':
                # Calculate the g'(z) values
                g_prime_z = self.activation_functions[i - 1][1](z_dict[f'layer {i}'])

                # Get dJ/da for each neuron a in layer i
                dJ_da = a_derivatives[f'layer {i}']

                # Find the element-wise product of g'(z) and dJ/da
                prod = (g_prime_z*dJ_da)

                # Reshape the product so that it can broadcasted with NumPy and be multiplied elementwise with the weight matrix
                prod = prod.reshape(input_layer.shape[0], 1, prod.shape[-1], order = 'C')

                # Save these derivatives in the dictionary for bias derivatives since this is it
                b_derivatives[f'layer {i}'] = prod
                
                # Get previous layer activations
                acts = a_dict[f'layer {i - 1}']

                # Reshape the previous layer activations to (n, p, 1) where n is the number of data points and p is the number of 
                # neurons in layer i - 1
                acts = acts.reshape(acts.shape[0], acts.shape[-1], 1, order = 'C')

                # Multiply the matrices elementwise, exploiting NumPy's broadcasting rules
                prod = prod*acts

                # Save these derivatives in the dictionary for weight derivatives
                w_derivatives[f'layer {i}'] = prod

            # Find dJ/dw and dJ/db via dJ/dz if layer i is softmax
            elif self.activation_function_name_list[i - 1] == 'softmax':
                # Get dJ/dz from the z_derivatives dictionary
                dJ_dz = z_derivatives[f'layer {i}'].reshape(input_layer.shape[0], 1, z_derivatives[f'layer {i}'].shape[-1], order = 'C')

                # Save these derivatives in the dictionary for bias derivatives
                b_derivatives[f'layer {i}'] = dJ_dz

                # Get dz/dW (which is just the activation for layer i - 1) from a_dict
                dz_dW = a_dict[f'layer {i - 1}'].reshape(input_layer.shape[0], a_dict[f'layer {i - 1}'].shape[-1], 1, order = 'C')

                # Multiply dJ/dz and dz/dW, exploiting numpy broadcasting rules to get an array of shape (n, p, q)
                prod = dJ_dz*dz_dW

                # Save these derivatives in the dictionary for weight derivatives
                w_derivatives[f'layer {i}'] = prod

        # Add the regularization derivatives to the derivatives of the cost w.r.t. each weight in the network
        for i in range(len(self.layers) - 1, 0, -1):
            w_derivatives[f'layer {i}'] += (lambd/n_training_set)*self.weights[f'layer {i}']

        # Return the dictionaries containing the derivatives
        return a_derivatives, w_derivatives, b_derivatives



    def gradient_check(self, input_row, true_label_row, loss, n_training_set, lambd=0, delta = 10**(-7)):
        # Compute the actual derivatives (we only need w_derivatives and b_derivatives)
        a_derivatives, w_derivatives, b_derivatives = self.derivatives(input_row, true_label_row, loss, n_training_set, lambd)

        # Create dictionaries to store the gradient estimates
        weight_grad_estimates = {}
        bias_grad_estimates = {}

        # Populate the gradient estmimate dictionaries with "zero" matrices of the same shape as the weight matrices for the respective layers
        for i in range(1, len(self.layers) - 1):
            weight_grad_estimates[f'layer {i}'] = np.zeros_like(self.weights[f'layer {i}'], dtype = float)
            bias_grad_estimates[f'layer {i}'] = np.zeros_like(self.biases[f'layer {i}'], dtype = float)

        # Iterate through weights and biases to compute the gradient estimates w.r.t. the individual weights and biases
        for i in range(1, len(self.layers) - 1):
            # Weights
            for j in range(self.weights[f'layer {i}'].shape[0]):
                for k in range(self.weights[f'layer {i}'].shape[1]):
                    # Modify the model's weight matrix to get two more weight matrices from which the gradient estimate will be calculated
                    w_delta_lower = self.weights[f'layer {i}'].copy()
                    w_delta_upper = self.weights[f'layer {i}'].copy()

                    w_delta_lower[j,k] -= delta
                    w_delta_upper[j,k] += delta

                    # Duplicate the weights of the model and modify them accordingly
                    w_delta_lower_full = self.weights.copy()
                    w_delta_lower_full[f'layer {i}'] = w_delta_lower
                    w_delta_upper_full = self.weights.copy()
                    w_delta_upper_full[f'layer {i}'] = w_delta_upper

                    # Compute the regularization sums
                    regularization_lower = (lambd/(2*n_training_set))*np.sum([np.sum(w_delta_lower_full[f'layer {ii}']**2) for ii in range(len(self.layers) - 1, 0, -1)])
                    regularization_upper = (lambd/(2*n_training_set))*np.sum([np.sum(w_delta_upper_full[f'layer {ii}']**2) for ii in range(len(self.layers) - 1, 0, -1)])

                    # Compute the costs and gradient estimate with regularization
                    cost_lower = np.sum(loss_function_dict[loss][0](true_label_row, self.call_with_parameters(input_row, w_delta_lower_full, self.biases))) + regularization_lower
                    cost_upper = np.sum(loss_function_dict[loss][0](true_label_row, self.call_with_parameters(input_row, w_delta_upper_full, self.biases))) + regularization_upper
                    derivative_estimate = (cost_upper - cost_lower)/(2*delta)

                    # Store the gradient estimate in the weight_grad_estimates dictionary
                    weight_grad_estimates[f'layer {i}'][j,k] = derivative_estimate

            # Biases
            for j in range(self.biases[f'layer {i}'].shape[1]):
                # Modify the model's weight matrix to get two more weight matrices from which the gradient estimate will be calculated
                b_delta_lower = self.biases[f'layer {i}'].copy()
                b_delta_upper = self.biases[f'layer {i}'].copy()

                b_delta_lower[0,j] -= delta
                b_delta_upper[0,j] += delta

                # Duplicate the biases of the model and modify them accordingly
                b_delta_lower_full = self.biases.copy()
                b_delta_lower_full[f'layer {i}'] = b_delta_lower
                b_delta_upper_full = self.biases.copy()
                b_delta_upper_full[f'layer {i}'] = b_delta_upper

                # Compute the regularization sums
                regularization_lower = (lambd/(2*n_training_set))*np.sum([np.sum(self.weights[f'layer {ii}']**2) for ii in range(len(self.layers) - 1, 0, -1)])
                regularization_upper = (lambd/(2*n_training_set))*np.sum([np.sum(self.weights[f'layer {ii}']**2) for ii in range(len(self.layers) - 1, 0, -1)])

                # Compute the costs and gradient estimate
                cost_lower = np.sum(loss_function_dict[loss][0](true_label_row, self.call_with_parameters(input_row, self.weights, b_delta_lower_full))) + regularization_lower
                cost_upper = np.sum(loss_function_dict[loss][0](true_label_row, self.call_with_parameters(input_row, self.weights, b_delta_upper_full))) + regularization_upper
                derivative_estimate = (cost_upper - cost_lower)/(2*delta)

                # Store the gradient estimate in the weight_grad_estimates dictionary
                bias_grad_estimates[f'layer {i}'][0,j] = derivative_estimate

        # Now all of the estimated gradients are stored. Next, all derivatives will be put into a long vector and gradient estimates will 
        # be put into a long vector. These will be compared to determine whether the computed derivative is the actual gradient.
        
        # Assemble the long vector for the computed derivative and the gradient estimate
        # Weights
        long_computed_derivatives_w = np.concatenate([w_derivatives[f'layer {i}'][0].reshape(-1,) for i in range(1, len(self.layers) - 1)])
        long_gradient_estimates_w = np.concatenate([weight_grad_estimates[f'layer {i}'].reshape(-1,) for i in range(1, len(self.layers) - 1)])

        # Biases
        long_computed_derivatives_b = np.concatenate([b_derivatives[f'layer {i}'][0].reshape(-1,) for i in range(1, len(self.layers) - 1)])
        long_gradient_estimates_b = np.concatenate([bias_grad_estimates[f'layer {i}'].reshape(-1,) for i in range(1, len(self.layers) - 1)])

        # Combination
        long_computed_derivatives = np.concatenate([long_computed_derivatives_w, long_computed_derivatives_b])
        long_gradient_estimates = np.concatenate([long_gradient_estimates_w, long_gradient_estimates_b])

        # Compute the gradient check value
        grad_check_value = np.linalg.norm(long_computed_derivatives - long_gradient_estimates)/(np.linalg.norm(long_computed_derivatives) + np.linalg.norm(long_gradient_estimates))

        # Print the gradient check value
        print(f'Gradient check value: {grad_check_value}')
        
        # Find cos(theta), where theta is the angle between the two derivative vectors in n-dimensional space.
        # If cos(theta) is near 1, then the angle between the derivative vectors is very close to 0, which indicates that they point in very similar directions.
        v = long_computed_derivatives
        w = long_gradient_estimates
        cos_theta = np.dot(v,w)/(np.linalg.norm(v)*np.linalg.norm(w))
        print(f'cos(theta)={cos_theta}\nv magnitude={np.linalg.norm(v)}, w magnitude={np.linalg.norm(w)}')
        return grad_check_value, cos_theta, (v==0).all(), (w==0).all()



    def step_size(self, optimizer, step_number, input_layer, true_labels, loss, n_training_set, lambd=0, previous_v = (0,0), previous_s = (0,0), beta_1 = 0.9, beta_2 = 0.999, epsilon = 10**(-8)):
        '''
        This function computes the step size for a given optimizer (before multiplying by the learning rate).
        
        Args:
            optimizer       :   Must be one of "sgd", "momentum", "rmsprop", or "adam".
            step_number     :   The number corresponding to how many updates have occurred since the of the v and s values
                                used in the optimizer calculations.
            input_layer     :   ndarray with shape (n, p) where p=self.layers[0], i.e., input_layer has the same shape 
                                as an input of the actual neural network.
            true_labels     :   ndarray with shape (n, q) where q=self.layers[-1], i.e., true_labels has the same shape 
                                as an output of the actual neural network.
            loss            :   The name of the type of loss to be used in the derivative calculation. For example, 
                                "mean squared error", "binary cross entropy", or "categorical cross entropy".
            previous_v      :   Tuple containing a dict containing the last v values computed for the "momentum" and 
                                "adam" optimizers for weights at index 0 and for biases at index 1.
            previous_s      :   Tuple containing a dict containing the last s values computed for the "rmsprop" and 
                                "adam" optimizers for weights at index 0 and for biases at index 1.
            beta_1          :   The constant beta used for the computation of v values for "momentum" and "adam".
            beta_2          :   The constant beta used for the computation of v values for "rmsprop" and "adam".
            epsilon         :   A small value to be added to denominators in "rmsprop" and "adam" calculations to prevent 
                                division by 0.
        '''
        # Calculate derivatives
        a_derivatives, w_derivatives, b_derivatives = self.derivatives(input_layer, true_labels, loss, n_training_set, lambd)

        # Get previous step dictionaries
        w_previous_v, b_previous_v = previous_v
        w_previous_s, b_previous_s = previous_s

        # Intialize dictionaries to store average gradients
        w_derivatives_avg = {}
        b_derivatives_avg = {}

        # Compute the average gradient for the minibatch (ideally this for loop would be vectorized)
        for i in range(len(self.layers) - 1, 0, -1):
            # Compute the average derivative of the cost w.r.t. the weights and biases in layer i, respectively
            dJ_dw_avg = np.mean(w_derivatives[f'layer {i}'], axis = 0)
            dJ_db_avg = np.mean(b_derivatives[f'layer {i}'], axis = 0)

            # Store average derivatives in their respective dictionaries
            w_derivatives_avg[f'layer {i}'] = dJ_dw_avg
            b_derivatives_avg[f'layer {i}'] = dJ_db_avg

        # Compute/select step sizes based on the optimizer input
        if (optimizer =='sgd'):
            step_sizes = w_derivatives_avg, b_derivatives_avg

            return step_sizes

        elif (optimizer == 'momentum'):
            # IMPLEMENT MOMENTUM WITH BIAS CORRECTION HERE

            bias_correction_value = 1 - beta_1**step_number

            # Initialize dictionaries to store the next v values
            v_dw = {}
            v_db = {}

            # Initialize dictionaries to store the next steps
            step_w = {}
            step_b = {}
            
            # Calculate the new steps v_dw and v_db for each layer i
            for i in range(len(self.layers) - 1, 0, -1):
                # Calculate the momentum terms
                v_dw[f'layer {i}'] = beta_1*w_previous_v[f'layer {i}'] + (1 - beta_1)*w_derivatives_avg[f'layer {i}']
                v_db[f'layer {i}'] = beta_1*b_previous_v[f'layer {i}'] + (1 - beta_1)*b_derivatives_avg[f'layer {i}']

                # Calculate the step sizes with bias correction
                step_w[f'layer {i}'] = v_dw[f'layer {i}']/bias_correction_value
                step_b[f'layer {i}'] = v_db[f'layer {i}']/bias_correction_value

            # Store the step_w, step_b, and v_dw, v_db dictionaries as tuples
            step_sizes = step_w, step_b
            v_values = v_dw, v_db

            return step_sizes, v_values

        elif (optimizer == 'rmsprop'):
            # IMPLEMENT RMSPROP WITH BIAS CORRECTION

            bias_correction_value = 1 - beta_2**step_number

            # Initialize dictionaries to store the next s values
            s_dw = {}
            s_db = {}

            # Initialize dictionaries to store the next steps
            step_w = {}
            step_b = {}
            
            # Calculate the new s_dw and s_db for each layer i then calculate the gradient step
            for i in range(len(self.layers) - 1, 0, -1):
                # Calculate the 'mean squares'
                s_dw[f'layer {i}'] = beta_2*w_previous_s[f'layer {i}'] + (1 - beta_2)*(w_derivatives_avg[f'layer {i}']**2)
                s_db[f'layer {i}'] = beta_2*b_previous_s[f'layer {i}'] + (1 - beta_2)*(b_derivatives_avg[f'layer {i}']**2)

                # Calculate the next steps with bias correction
                step_w[f'layer {i}'] = w_derivatives_avg[f'layer {i}']/(np.sqrt(s_dw[f'layer {i}']/bias_correction_value) + epsilon)
                step_b[f'layer {i}'] = b_derivatives_avg[f'layer {i}']/(np.sqrt(s_db[f'layer {i}']/bias_correction_value) + epsilon)
                
                if (s_db[f'layer {i}']**0.5 < 0).any():
                    print('WHAT')


            
            # Organize the step sizes and s values into tuples before returning them
            step_sizes = step_w, step_b
            s_values = s_dw, s_db

            return step_sizes, s_values


        elif (optimizer == 'adam'):
            # IMPLEMENT ADAM OPTIMIZER WITH BIAS CORRECTION

            bias_correction_value_1 = 1 - beta_1**step_number
            bias_correction_value_2 = 1 - beta_2**step_number

            # Initialize dictionaries to store the next v values
            v_dw = {}
            v_db = {}

            # Initialize dictionaries to store the next s values
            s_dw = {}
            s_db = {}

            # Initialize dictionaries to store the next steps
            step_w = {}
            step_b = {}
            
            # Calculate the new steps v_dw and v_db for each layer i
            for i in range(len(self.layers) - 1, 0, -1):
                # Calculate the momentum terms
                v_dw[f'layer {i}'] = beta_1*w_previous_v[f'layer {i}'] + (1 - beta_1)*w_derivatives_avg[f'layer {i}']
                v_db[f'layer {i}'] = beta_1*b_previous_v[f'layer {i}'] + (1 - beta_1)*b_derivatives_avg[f'layer {i}']

                # Calculate the 'mean squares'
                s_dw[f'layer {i}'] = beta_2*w_previous_s[f'layer {i}'] + (1 - beta_2)*(w_derivatives_avg[f'layer {i}']**2)
                s_db[f'layer {i}'] = beta_2*b_previous_s[f'layer {i}'] + (1 - beta_2)*(b_derivatives_avg[f'layer {i}']**2)

                # Calculate the next steps with bias corrections
                step_w[f'layer {i}'] = (v_dw[f'layer {i}']/bias_correction_value_1)/(np.sqrt(s_dw[f'layer {i}']/bias_correction_value_2) + epsilon)
                step_b[f'layer {i}'] = (v_db[f'layer {i}']/bias_correction_value_1)/(np.sqrt(s_db[f'layer {i}']/bias_correction_value_2) + epsilon)
                
                if (s_db[f'layer {i}'] < 0).any():
                    print('WHAT')

            # Organize the step sizes, v values and s values into tuples before returning them
            step_sizes = step_w, step_b
            v_values = v_dw, v_db
            s_values = s_dw, s_db

            return step_sizes, v_values, s_values



    def gradient_descent(self, alpha, batch_size, epochs, optimizer, input_layer, true_labels, loss, lambd, cv_input_layer = None, cv_true_labels = None, beta_1 = 0.9, beta_2 = 0.999, epsilon = 10**(-8)):
        '''
        This function applies gradient descent to the weights and biases of the "neural" instance. 

        Args:
            alpha           :   The learning rate. Must be a float or int.
            batch_size      :   The number of data points from which a gradient step will be calculated. Must be a 
                                positive int.
            epochs          :   The number of times to go over the whole dataset to perform the gradient descent. Must be 
                                a positive int.
            optimizer       :   The optimizer to be used in calculations. Must be one of "sgd", "momentum", "rmsprop", 
                                or "adam".
            input_layer     :   ndarray with shape (n, p) where p=self.layers[0], i.e., input_layer has the same shape 
                                as an input of the actual neural network.
            true_labels     :   ndarray with shape (n, q) where q=self.layers[-1], i.e., true_labels has the same shape 
                                as an output of the actual neural network.
            loss            :   The name of the type of loss to be used in the derivative calculation. For example, 
                                "mean squared error", "binary cross entropy", or "categorical cross entropy".
            lambd           :   const. The regularization lambda value. This value should be positive.
            cv_input_layer  :   ndarray with shape (n_1, p) where p=self.layers[0], i.e., cv_input_layer has the same shape 
                                as an input of the actual neural network. If None, then no cv_accuracy or cv_loss will be printed or added to the history.
            cv_true_labels  :   ndarray with shape (n_1, q) where q=self.layers[-1], i.e., cv_true_labels has the same shape 
                                as an output of the actual neural network. If None, then no cv_accuracy or cv_loss will be printed or added to the history.
            previous_v      :   Tuple containing a dict containing the last v values computed for the "momentum" and 
                                "adam" optimizers for weights at index 0 and for biases at index 1.
            previous_s      :   Tuple containing a dict containing the last s values computed for the "rmsprop" and 
                                "adam" optimizers for weights at index 0 and for biases at index 1.
            beta_1          :   The constant beta used for the computation of v values for "momentum" and "adam".
            beta_2          :   The constant beta used for the computation of v values for "rmsprop" and "adam".
            epsilon         :   A small value to be added to denominators in "rmsprop" and "adam" calculations to prevent 
                                division by 0.
        '''
        # Create dictionaries to store v_values and s_values for both the weights and biases
        w_v_values = {}
        b_v_values = {}
        
        w_s_values = {}
        b_s_values = {}

        # Initialize the v_dw, v_db, s_dw and s_db to 0 for each layer i
        for i in range(len(self.layers) - 1, 0, -1):
            w_v_values[f'layer {i}'] = np.zeros_like(self.weights[f'layer {i}'], dtype=float)
            b_v_values[f'layer {i}'] = np.zeros_like(self.biases[f'layer {i}'], dtype=float)

            w_s_values[f'layer {i}'] = np.zeros_like(self.weights[f'layer {i}'], dtype=float)
            b_s_values[f'layer {i}'] = np.zeros_like(self.biases[f'layer {i}'], dtype=float)

        # Initialize the step number t which will be used to calculate the bias correction term (1-beta^t) for the optimizers
        step = 1

        for i in range(epochs):
            # Shuffle the data (CHECK THIS STEP TO SEE IF IT RUINS THE GRADIENT DESCENT)
            permutation = np.random.permutation(input_layer.shape[0])
            xx = input_layer[permutation]
            yy = true_labels[permutation]

            # Perform minibatch gradient descent using minibatches of size batch_size
            for j in range(-(-len(xx)//batch_size)):
                # Get a minibatch
                minibatch_x = xx[j : min(j + batch_size, xx.shape[0])]
                minibatch_y = yy[j : min(j + batch_size, yy.shape[0])]

                previous_v_ = w_v_values, b_v_values
                previous_s_ = w_s_values, b_s_values

                # Calculate the gradient steps, v_values and s_values
                step_sizes_output = self.step_size(optimizer, step, minibatch_x, minibatch_y, loss, input_layer.shape[0], lambd, previous_v = previous_v_, previous_s = previous_s_, beta_1 = beta_1, beta_2 = beta_2, epsilon = epsilon)
                
                # Update the step number for the next update
                step += 1

                # Separate the actual step sizes from the rest of the outputs in step_sizes_outputs
                if optimizer == 'sgd':
                    w_step_sizes, b_step_sizes = step_sizes_output
                elif optimizer == 'momentum':
                    w_step_sizes, b_step_sizes = step_sizes_output[0]
                    w_v_values, b_v_values = step_sizes_output[1]
                elif optimizer == 'rmsprop':
                    w_step_sizes, b_step_sizes = step_sizes_output[0]
                    w_s_values, b_s_values = step_sizes_output[1]
                elif optimizer == 'adam':
                    w_step_sizes, b_step_sizes = step_sizes_output[0]
                    w_v_values, b_v_values = step_sizes_output[1]
                    w_s_values, b_s_values = step_sizes_output[2]

                # Update the weights and biases
                for k in range(len(self.layers) - 1, 0, -1):
                    self.weights[f'layer {k}'] -= alpha * w_step_sizes[f'layer {k}']
                    self.biases[f'layer {k}'] -= alpha * b_step_sizes[f'layer {k}']

                # Every so often, store the history of the weights and biases
                if j%10 == 0:
                    # Initialize list to store max absolute gradients for each layer to see whether gradients are vanishing or exploding
                    w_grad_max = []
                    for k in range(len(self.layers) - 1, 0, -1):
                        w_grad_max.insert(0, np.max(np.abs(w_step_sizes[f'layer {k}'])))

                    # Initialize list to store max absolute gradients for each layer to see whether gradients are vanishing or exploding
                    w_grad_mean = []
                    for k in range(len(self.layers) - 1, 0, -1):
                        w_grad_mean.insert(0, np.mean(np.abs(w_step_sizes[f'layer {k}'])))

                    # First get the model predictions
                    output_layer = self.__call__(input_layer)
                    
                    # Compute the cost and accuracy
                    cost = np.mean(loss_function_dict[loss][0](true_labels, output_layer))
                    acc = self.accuracy(true_labels, input_layer)

                    # Save the history
                    self.cost_history = np.append(self.cost_history, cost)
                    self.accuracy_history = np.append(self.accuracy_history, acc)
                    self.weight_update_max_history = np.vstack([self.weight_update_max_history, w_grad_max])
                    self.weight_update_mean_history = np.vstack([self.weight_update_mean_history, w_grad_mean])

                    # Compute and save the cross-validation cost and accuracy
                    if (type(cv_input_layer) != type(None)) and (type(cv_true_labels) != type(None)):
                        # Get neural network prediction
                        cv_output_layer = self.__call__(cv_input_layer)

                        # Compute cost and accuracy
                        cv_cost = np.mean(loss_function_dict[loss][0](cv_true_labels, cv_output_layer))
                        cv_acc = self.accuracy(cv_true_labels, cv_input_layer)

                        # Save in the dictionaries
                        self.cv_cost_history = np.append(self.cv_cost_history, cv_cost)
                        self.cv_accuracy_history = np.append(self.cv_accuracy_history, cv_acc)

                # Every so often, print feedback
                if j%10 == 0:
                    # Initialize list to store max absolute gradients for each layer to see whether gradients are vanishing or exploding
                    w_grad_max = []
                    for k in range(len(self.layers) - 1, 0, -1):
                        w_grad_max.insert(0, np.max(np.abs(w_step_sizes[f'layer {k}'])))

                    # Compute and print the average loss at the end of the epoch
                    # First get the model predictions
                    output_layer = self.__call__(input_layer)

                    # Compute the cost
                    cost = np.mean(loss_function_dict[loss][0](true_labels, output_layer))

                    # Compute the accuracy
                    acc = self.accuracy(true_labels, input_layer)
                    
                    # Print the stats
                    print(f'Max absolute weight updates for each layer: {w_grad_max}\nUnregularized cost at minibatch {j} of epoch {i} : {cost}\nAccuracy at minibatch {j} epoch {i} : {acc}')
                        
            # Print the unregularized cost
            if (i%1 == 0) or (i == epochs - 1):
                # Compute and print the average loss at the end of the epoch
                # First get the model predictions
                predictions = self.__call__(input_layer)

                # Compute the cost
                cost = np.mean(loss_function_dict[loss][0](true_labels, predictions))

                print(f'Unregularized cost at epoch {i} : {cost}')

                # Compute the accuracy
                acc = self.accuracy(true_labels, input_layer)
                print(f'Accuracy at epoch {i} : {acc}')


# Engineer the Data

Since the data consists of handrwritten digits, the outputs are categorical. Thus, a softmax output layer will be ideal and the labels must be one-hot encoded.

In [ ]:
# Data to be used to train the model
cutoff = 10000

# Load data (you need to download the relevant dataset as a CSV, e.g. CIFAR-10, MNIST digits, or Fashion-MNIST)
data = pd.read_csv("path_to_data", nrows=cutoff*2)

# Separate inputs and labels
x_train = data[data.columns[:-1]]
y_train = data[[data.columns[-1]]]

# Put the data into arrays
x_array = np.array(x_train)
y_array = np.array(y_train)

# Scale inputs by 1/255
x_array = x_array/255

# One-hot encoding of labels
y_one_hot = np.array([[1 if y_array[i, 0] == j else 0 for j in range(10)] for i in range(y_array.shape[0])])

# Data to be used to train the model
xxx = x_array[:cutoff]
yyy = y_one_hot[:cutoff]

# Cross-validation data
xxx_cv = x_array[cutoff:cutoff*2]
yyy_cv = y_one_hot[cutoff:cutoff*2]


# Train a Neural Network

In [ ]:
# Select parameters for gradient descent
learning_rate_ = 0.001
minibatch_size_ = 100
epochs_ = 20
optimizer_ = 'adam'
inputs_ = xxx
true_outputs_ = yyy
loss_ = 'categorical cross entropy'
cv_input_layer_ = xxx_cv
cv_true_labels_ = yyy_cv
lambd_ = 0
beta_1_ = 0.9
beta_2_ = 0.999
epsilon_ = 10**(-8)

# Select network architecture
# Note that the activation functions are only for the layers after the first layer since the first layer is the input layer
# Therefore, the activation function list must be one less than the layer dimensions list
layer_dimensions = [xxx.shape[1], 128, 128, 128, 128, yyy.shape[1]]
activation_functions = ['leaky relu', 'leaky relu', 'leaky relu', 'leaky relu', 'softmax']

# Initialize neural network without
nn = neural(layers=layer_dimensions, activation_function_list=activation_functions, he_uniform=True)

In [ ]:
# Run gradient descent on the network
nn.gradient_descent(learning_rate_, minibatch_size_, epochs_, optimizer_, inputs_, true_outputs_, loss_, lambd_, cv_input_layer = cv_input_layer_, cv_true_labels = cv_true_labels_, beta_1 = beta_1_, beta_2 = beta_2_, epsilon = epsilon_)

# Train Models for an Ablation Study

In [8]:
# Create lists of the variables to be included in the ablation study
optimizer_choice = ['sgd', 'momentum', 'rmsprop', 'adam']
optimizer_choice_lr = [0.01, 0.03, 0.001, 0.001] # These are not variables, but are the fixed learning rates associated with each optimizer
he_initial_choice = [False, True]
regularization_choice = [0, 1, 2, 3]

In [ ]:
# Train the models for the ablation study

import pickle

# Select the name of the file that the dictionary will be store in
file_path = 'neural_dict.pkl'

# Initialize the dictionary that will store all of the neural networks
try:
    with open(file_path, 'rb') as file:
        neural_network_dict = pickle.load(file)
except:
    neural_network_dict = {}

# Load the last index started
try:
    checkpoint_index = np.load('checkpoint_index_i.npy')[0]
except:
    checkpoint_index = 0


# Continue from the last saved checkpoint
for i in range(checkpoint_index, len(optimizer_choice)):
    # Save the current index so that progress can be resumed from a checkpoint
    np.save('checkpoint_index_i.npy', np.array([i]))

    # Initialize dictionary to store the neural networks
    neural_network_dict[optimizer_choice[i]] = {}

    # Loop over the rest of the variables
    for j in range(len(he_initial_choice)):
        neural_network_dict[optimizer_choice[i]][f'He Uniform {he_initial_choice[j]}'] = {}
        for k in range(len(regularization_choice)):
            # Select parameters for gradient descent
            learning_rate_ = optimizer_choice_lr[i]
            minibatch_size_ = 100
            epochs_ = 1
            optimizer_ = optimizer_choice[i]
            inputs_ = xxx
            true_outputs_ = yyy
            loss_ = 'categorical cross entropy'
            lambd_ = regularization_choice[k]
            cv_input_layer_ = xxx_cv
            cv_true_labels_ = yyy_cv
            beta_1_ = 0.9
            beta_2_ = 0.999
            epsilon_ = 10**(-8)
            
            # Select network structure
            layer_dimensions = [xxx.shape[1], 128, 128, 128, 128, yyy.shape[1]]
            activation_functions = ['leaky relu', 'leaky relu', 'leaky relu', 'leaky relu', 'softmax']

            # Initialize neural network with the choice of He initialization or not
            nn = neural(layers=layer_dimensions, activation_function_list=activation_functions, he_uniform=he_initial_choice[j])

            neural_network_dict[optimizer_choice[i]][f'He Uniform {he_initial_choice[j]}'][f'Regularization lambd {regularization_choice[k]}'] = nn

            # Train neural network
            nn.gradient_descent(learning_rate_, minibatch_size_, epochs_, optimizer_, inputs_, true_outputs_, loss_, lambd_, cv_input_layer=cv_input_layer_, cv_true_labels=cv_true_labels_, beta_1 = beta_1_, beta_2 = beta_2_, epsilon = epsilon_)

            # Update the file containing the dictionary
            with open(file_path, 'wb') as file:
                pickle.dump(neural_network_dict, file)

# Load Models for the Ablation Study

In [ ]:
# Put the neural_network_dictionary into a numpy array to enable easy isolation of variables to conduct an ablation study effectively
import pickle

# Load the dictionary containing the neural networks
file_path = 'neural_dict.pkl'
with open(file_path, 'rb') as file:
    neural_dict = pickle.load(file)

In [ ]:
# Initialize an array for objects that contains the string 'a' as placeholders
neural_array = np.full(shape=(len(optimizer_choice), len(he_initial_choice), len(regularization_choice)), fill_value='a', dtype=object)
'''
Indices (from left most index):

    - Optimizer choice: "sgd" [0], "momentum" [1], "rmsprop" [2], "adam" [3]
    - He initialization choice: False [0], True [1]
    - Regularization choice: lambd=0 [0], lambd=1 [1], lambd=2 [2], lambd=3 [3]
'''

# Populate the neural_array with the relevant neural network at each index
for i in range(len(optimizer_choice)):
    for j in range(len(he_initial_choice)):
        for k in range(len(regularization_choice)):
            neural_array[i,j,k] = neural_dict[optimizer_choice[i]][f'He Uniform {he_initial_choice[j]}'][f'Regularization lambd {regularization_choice[k]}']